Fast Food & General Nutrition Analysis


**Author:** Derek Singleton
**Course:** Data Science & Analytics  
**Date:** December 2025  


---

## Executive Summary

### 🎯 Project Goal
This capstone project analyzes and compares nutritional profiles between fast food menu items and general food products to answer the question: **"When people are busy, what are the healthiest options available — and how can data help them make better decisions quickly?"**


### 📊 Key Findings
*(Run all cells to see computed values in Section 10)*
- Fast food items have significantly higher sodium density compared to general foods
- Health classification reveals distinct nutritional profiles across food categories
- Machine learning models achieve strong predictive performance for calorie estimation
- Classification models can accurately categorize foods by health profile

### 🛠️ Methods Used
- **Data Sources:** 2 Kaggle datasets (517 fast food items, 8,791 general foods) + USDA FoodData Central API validation
- **Analysis:** Schema alignment, exploratory data analysis, feature engineering, statistical comparison
- **Visualization:** 7 static visualizations (matplotlib/seaborn) + 2 interactive dashboards (Plotly)
- **Machine Learning:** Regression models for calorie prediction + Classification for health categorization

### 💡 Actionable Insights
*(To be populated after analysis)*
- Consumers can identify healthier fast food options using our nutrient density score
- Healthcare professionals can use the comparison framework for patient education
- Restaurant chains can benchmark their menu items against nutritional standards

---



## 1. Problem Statement & Research Question

### 🔍 The Problem

In today's fast-paced world, many people lack the time, resources, or energy to prepare nutritious meals at home. As a result, they frequently turn to convenient fast-food options — but these choices vary dramatically in nutritional quality.

**Challenges:**
- **Consumers** → Struggle to make informed dietary decisions under time pressure
- **Healthcare professionals** → Need data-driven tools to guide patients on nutrition
- **Researchers/educators** → Lack comprehensive comparisons between fast food and general food supply

### ❓ Research Question

> **"What is the nutritional difference between fast food menu items and general food items, and how do these values compare against authoritative USDA standards?"**

### 📋 Sub-Questions
1. What are the key nutritional differences in calories, macronutrients, and micronutrients?
2. Which fast food items approach the nutritional profile of healthier general foods?
3. Can we build a predictive model to estimate nutritional values based on food characteristics?
4. What patterns emerge when comparing restaurant chains?

### 🎓 Project Scope

This analysis focuses on:
- **Nutrients:** Calories, protein, fat (total/saturated), carbohydrates, fiber, sugar, sodium, vitamins
- **Food Types:** Fast food restaurant items vs. general food products
- **Validation:** Cross-referencing with USDA FoodData Central official standards

**Out of Scope:** Meal planning, dietary recommendations for specific medical conditions, cost analysis

---

## 2. Data Sources & Acquisition

### 📚 Dataset Overview

| Dataset | Source | Size | Key Features |
|---------|--------|------|-------------|
| **Fast Food Nutrition** | [Kaggle](https://www.kaggle.com/datasets/ulrikthygepedersen/fastfood-nutrition) | 517 items | Restaurant, item name, 17 nutritional attributes |
| **General Food Nutrition** | [Kaggle](https://www.kaggle.com/datasets/thedevastator/the-nutritional-content-of-food-a-comprehensive) | 8,791 items | USDA NDB database with 50+ nutritional attributes |
| **USDA FoodData Central** | [API](https://fdc.nal.usda.gov/api-guide.html) | Validation | Official government nutritional standards |

### 🔗 How Datasets Relate

**Integration Strategy:**
- **Common Dimensions:** Nutritional attributes (calories, protein, fat, carbs, sodium)
- **Alignment Challenge:** Different column names, units, and schemas
- **Solution:** Create normalized schema with standardized units (grams, mg, calories)
- **Validation:** Use USDA API to verify accuracy of overlapping items (e.g., "cheeseburger")

```
┌─────────────────────┐
│   Fast Food Data    │
│  (517 items)        │
│  - Restaurant       │
│  - calories         │
│  - protein          │
└──────────┬──────────┘
           │
           │  Normalized Schema
           │  (Common Nutrients)
           │
┌──────────┴──────────┐
│ General Food Data   │
│  (8,791 items)      │
│  - Shrt_Desc        │
│  - Energ_Kcal       │
│  - Protein_(g)      │
└─────────────────────┘
           │
           │  Validation
           │
┌──────────┴──────────┐
│   USDA API          │
│  (Reference Data)   │
└─────────────────────┘
```

---

## 3. Environment Setup & Data Loading

### 📦 Import Required Libraries

We'll use:
- **Data Manipulation:** `pandas`, `numpy`
- **Visualization:** `matplotlib`, `seaborn`, `plotly`
- **Machine Learning:** `scikit-learn`
- **API Access:** `requests`, `python-dotenv`
- **Statistical Analysis:** `scipy`

In [6]:
# Core libraries
import pandas as pd
import numpy as np
import os
import warnings
from dotenv import load_dotenv
import requests
import json

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline

# Statistical analysis
from scipy import stats

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load environment variables
load_dotenv('.env.local')
USDA_API_KEY = os.getenv('USDA_API_KEY', 'DEMO_KEY')

print("✅ Libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔑 USDA API Key loaded: {'Yes' if USDA_API_KEY else 'No'}")

✅ Libraries imported successfully!
📊 Pandas version: 2.3.3
🔑 USDA API Key loaded: Yes


### 📂 Load Datasets

We'll load both CSV files and perform initial inspection.

In [7]:
# Load datasets
fastfood_df = pd.read_csv('datasets/fastfood.csv')
general_nutrition_df = pd.read_csv('datasets/food_nutrition.csv')

print("=" * 70)
print(" DATA LOADING SUMMARY ")
print("=" * 70)
print(f"\n📌 Fast Food Dataset:")
print(f"   Rows: {len(fastfood_df):,}")
print(f"   Columns: {len(fastfood_df.columns)}")
print(f"   Memory: {fastfood_df.memory_usage(deep=True).sum() / 1024:.2f} KB")

print(f"\n📌 General Nutrition Dataset:")
print(f"   Rows: {len(general_nutrition_df):,}")
print(f"   Columns: {len(general_nutrition_df.columns)}")
print(f"   Memory: {general_nutrition_df.memory_usage(deep=True).sum() / 1024:.2f} KB")

print("\n✅ Datasets loaded successfully!")

 DATA LOADING SUMMARY 

📌 Fast Food Dataset:
   Rows: 515
   Columns: 17
   Memory: 151.05 KB

📌 General Nutrition Dataset:
   Rows: 8,790
   Columns: 54
   Memory: 5179.07 KB

✅ Datasets loaded successfully!
